In [1]:
import os
import random

import pandas as pd
from config import SETTINGS
from pysld.style import StyleSld
from sqlalchemy import text

import seabeepy as sb

# Create standardised SLDs

GeoNode and GeoServer use `.sld` files for layer styling. This notebook takes the colour schemes defined in the class definitions files (`.ecs`) for mapping habitats and converts them to SLD files. This means layers published on GeoNode can use the same colour schemes defined by ecologists for habitat mapping.

## 1. Simple styles

In [2]:
out_fold = r"/home/notebook/annotation/sld_files"

In [3]:
# Dict mapping file names to fill and stroke colours
# name => (fill_colour, stroke_colour)
style_dict = {
    "black_outline": (None, "#000000"),
    "red_outline": (None, "#FF0000"),
    "blue_outline": (None, "#0000FF"),
}
for name, (fill, stroke) in style_dict.items():
    sld = StyleSld(
        style_name=name,
        geom_type="polygon",
        fill_color=fill,
        stroke_color=stroke,
    )
    sld_style = sld.generate_simple_style()
    sld_path = os.path.join(out_fold, f"{name}.sld")
    with open(sld_path, "w") as f:
        f.write(sld_style)

## 2. Styles for habitat survey missions

In [4]:
versions = ["0-1", "0-2", "0-4", "1-0"]
for version in versions:
    df = sb.anno.get_class_codes(version)
    del df["desc"]
    for level in [1, 2, 3]:
        code_len = level * 2
        lev_df = df[df["code"].str.len() == code_len].drop_duplicates()

        sld = StyleSld(
            style_name=f"annotation_v{version}_level{level}",
            geom_type="polygon",
            attribute_name=f"lev{level}_name",
            values=lev_df["name"].tolist(),
            color_palette=lev_df["colour"].tolist(),
            opacity=0.5,
        )
        sld_style = sld.generate_categorized_style()
        sld_path = os.path.join(
            out_fold, f"annotation_classes_v{version}_level{level}.sld"
        )
        with open(sld_path, "w") as f:
            f.write(sld_style)

## 3. Styles for seabrids detection

**The code below is not currently used** but it might be useful in the future. PySLD doesn't seem to support varying just polygon outline colours, but the function below should work instead. Right now, I think it's better to choose a simple style for the seabirds results.

In [5]:
# def generate_sld(attribute_name, style_name, values, single_color=None, opacity=1):
#     sld = f"""<?xml version="1.0" encoding="ISO-8859-1"?>
# <sld:StyledLayerDescriptor xmlns="http://www.opengis.net/sld" xmlns:sld="http://www.opengis.net/sld" xmlns:ogc="http://www.opengis.net/ogc" xmlns:gml="http://www.opengis.net/gml" version="1.0.0">
#   <sld:NamedLayer>
#     <sld:Name>{style_name}</sld:Name>
#     <sld:UserStyle>
#       <sld:Name>{style_name}</sld:Name>
#       <sld:FeatureTypeStyle>"""

#     for value in values:
#         # If a single color is not specified, generate a random color for each value
#         color = (
#             single_color
#             if single_color
#             else "#{:06x}".format(random.randint(0, 0xFFFFFF))
#         )

#         sld += f"""
#         <sld:Rule>
#           <sld:Title>{value}</sld:Title>
#           <ogc:Filter>
#             <ogc:PropertyIsEqualTo>
#               <ogc:PropertyName>{attribute_name}</ogc:PropertyName>
#               <ogc:Literal>{value}</ogc:Literal>
#             </ogc:PropertyIsEqualTo>
#           </ogc:Filter>
#           <sld:PolygonSymbolizer>
#             <sld:Fill>
#               <sld:CssParameter name="fill">{color}</sld:CssParameter>
#               <sld:CssParameter name="fill-opacity">{opacity}</sld:CssParameter>
#             </sld:Fill>
#             <sld:Stroke>
#               <sld:CssParameter name="stroke">{color}</sld:CssParameter>
#               <sld:CssParameter name="stroke-width">1</sld:CssParameter>
#             </sld:Stroke>
#           </sld:PolygonSymbolizer>
#         </sld:Rule>"""

#     sld += """
#       </sld:FeatureTypeStyle>
#     </sld:UserStyle>
#   </sld:NamedLayer>
# </sld:StyledLayerDescriptor>"""

#     return sld

In [6]:
# eng = sb.storage.connect_postgis(SETTINGS.DB_USER, SETTINGS.DB_PASSWORD)
# tables = ["species", "activity", "sex", "age"]
# for table in tables:
#     sql = text(f"SELECT * FROM {table}")
#     df = pd.read_sql(sql, eng).add_prefix(f"{table}_")

#     if f"{table}_norwegian" in df.columns:
#         idx_col = f"{table}_norwegian"
#     else:
#         idx_col = f"{table}_description"

#     sld_style = generate_sld(
#         f"{table}_description",
#         f"seabirds_{table}",
#         df[idx_col].tolist(),
#         single_color=None,
#         opacity=0,
#     )
#     sld_path = os.path.join(out_fold, f"seabrids_{table}.sld")
#     with open(sld_path, "w") as f:
#         f.write(sld_style)